In [1]:
import warnings
warnings.filterwarnings('ignore')
import os

import tws_data 
import statsmodels.tsa.stattools as ts
import statsmodels.api as sm
from ib_insync import *
import numpy as np
import pandas as pd
import datetime as dt
import time
pd.options.display.float_format = '{:,.4f}'.format
%matplotlib inline
from scipy.stats import norm as norm
import matplotlib.pyplot as plt
import requests as req
from bs4 import BeautifulSoup as bs
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import math

def time_block_strtoactual(i):
    curr_time_block = dt.datetime.today().replace(hour = 9, minute = 30, second = 0, microsecond = 0) + dt.timedelta(minutes = 30*(i-1))
    return curr_time_block.strftime("%H:%M")

In [2]:
os.chdir('D:\Options Data\IB Intraday')

def create_dfrow(curr_option):
    df_row = pd.DataFrame({'Symbol': curr_option.contract.localSymbol, 
                           'Type':curr_option.contract.right, 
                           'Bid': curr_option.bid, 
                           'Ask': curr_option.ask, 
                           'Volume': curr_option.volume, 
                           'Strike': curr_option.contract.strike,
                           'Expiry': dt.datetime.strptime(curr_option.contract.lastTradeDateOrContractMonth, '%Y%m%d'),
                           'bidIV': curr_option.bidGreeks.impliedVol, 
                           'bidDelta': curr_option.bidGreeks.delta,
                           'bidGamma': curr_option.bidGreeks.gamma, 
                           'bidVega': curr_option.bidGreeks.vega,
                           'bidTheta': curr_option.bidGreeks.theta, 
                           'askIV': curr_option.askGreeks.impliedVol, 
                           'askDelta': curr_option.askGreeks.delta,
                           'askGamma': curr_option.askGreeks.gamma, 
                           'askVega': curr_option.askGreeks.vega,
                           'askTheta': curr_option.askGreeks.theta,
                           'modelIV': curr_option.modelGreeks.impliedVol, 
                           'modelDelta': curr_option.modelGreeks.delta,
                           'modelGamma': curr_option.modelGreeks.gamma, 
                           'modelVega': curr_option.modelGreeks.vega,
                           'modelTheta': curr_option.modelGreeks.theta}, index = [0])
    return df_row

def time_remaining(weekly_options, curr_spx):
    typ = weekly_options.reset_index().loc[0,'Type']

    try:
        interest_rate = bs(req.get('https://www.marketwatch.com/investing/bond/tmubmusd01m?countrycode=bx','lxml').text, 'lxml')
        interest_rate = float(interest_rate.select_one('h3[class*="intraday__price"]').select_one('bg-quote').text)/100
    except:
        interest_rate = 0.02422

    weekly_options['a'] = interest_rate - (weekly_options['modelIV']**2)/2

    if typ == 'P':
        weekly_options['b'] = weekly_options['modelIV']*norm.ppf(-weekly_options['modelDelta'])
    if typ == 'C':
        weekly_options['b'] = -weekly_options['modelIV']*norm.ppf(weekly_options['modelDelta'])
    weekly_options['c'] = np.log(curr_spx/weekly_options['Strike'])

    weekly_options_otm = weekly_options[abs(weekly_options.modelDelta) <= 0.5]
    weekly_options_itm = weekly_options[abs(weekly_options.modelDelta) > 0.5]

    if typ == 'P':
        weekly_options_otm['time_remaining'] = ((-weekly_options_otm.b - 
                                                 np.sqrt(weekly_options_otm.b**2 - 
                                                         4*weekly_options_otm.a*weekly_options_otm.c))/(2*weekly_options_otm.a))**2
        weekly_options_itm['time_remaining'] = ((-weekly_options_itm.b + 
                                                 np.sqrt(weekly_options_itm.b**2 - 
                                                         4*weekly_options_itm.a*weekly_options_itm.c))/(2*weekly_options_itm.a))**2
    if typ == 'C':
        weekly_options_otm['time_remaining'] = ((-weekly_options_otm.b + 
                                                 np.sqrt(weekly_options_otm.b**2 - 
                                                         4*weekly_options_otm.a*weekly_options_otm.c))/(2*weekly_options_otm.a))**2
        weekly_options_itm['time_remaining'] = ((-weekly_options_itm.b - 
                                                 np.sqrt(weekly_options_itm.b**2 - 
                                                         4*weekly_options_itm.a*weekly_options_itm.c))/(2*weekly_options_itm.a))**2
    weekly_times = pd.concat([weekly_options_otm, 
                              weekly_options_itm], axis = 0)[['Expiry','time_remaining']].dropna().groupby('Expiry').mean()

    del weekly_options['a'], weekly_options['b'], weekly_options['c'], weekly_options_itm, weekly_options_otm

    return weekly_options.merge(weekly_times.reset_index(), on = 'Expiry')


def get_options(ticker, exchange, contractType, earliest_date, latest_date, curr_iv, stdevs, save_csv = False):
    if contractType == 'Stock':
        security = Stock(ticker,exchange)
    else:
        security = Index(ticker, exchange)

    ib.qualifyContracts(security)

    chains = ib.reqSecDefOptParams(security.symbol, '', security.secType, security.conId)

    chains_df = util.df(chains)
    chain = next(c for c in chains if c.tradingClass == ticker and c.exchange == 'SMART')

    all_expirations = sorted(exp for exp in chain.expirations)
    expirations_dates = [dt.datetime.strptime(x, '%Y%m%d') for x in all_expirations]

    expirations = list(filter(lambda x: x[0] >= earliest_date and x[0] <= latest_date, list(zip(expirations_dates,all_expirations))))
    dte_expirations = [(expir[0] - dt.datetime.today()).days for expir in expirations]
    expirations = [expir[1] for expir in expirations]

    ib.reqHeadTimeStamp(security, whatToShow='TRADES', useRTH=True)

    end_date = '' #'20100506 13:00:00 PST'
    duration = '1 D'

    sec_bars = ib.reqHistoricalData(
            security,
            endDateTime=end_date,
            durationStr=duration,
            barSizeSetting='1 min',
            whatToShow='TRADES',
            useRTH=True,
            formatDate=1)


    sec_df = util.df(sec_bars).set_index('date')
    sec_df.index = pd.to_datetime(sec_df.index)

    curr_sec_price = sec_df.reset_index().loc[len(sec_df) - 1, 'close']
    bounds_lst = [curr_sec_price*(stdevs*stdevs/(100*np.sqrt(365.25/dte))) for dte in dte_expirations]


    contracts_list = []

    for bounds, expiration in zip(bounds_lst, expirations):
        strikes = [strike for strike in chain.strikes
                   if strike % 5 == 0
                   and curr_sec_price - bounds < strike < curr_sec_price + bounds]

        rights = ['P', 'C']

        curr_contracts = [Option(ticker, expiration, strike, right, 'SMART')
                          for right in rights
                          for strike in strikes]
        contracts_list += [curr_contracts]

    puts_list = []
    calls_list = []

    for contracts in contracts_list:
        ib.qualifyContracts(*contracts)
        sec_options = ib.reqTickers(*contracts)

        puts_rows = []
        calls_rows = []

        for curr_option in sec_options:
            try:
                curr_row = create_dfrow(curr_option)
                if curr_option.contract.right == 'P':
                    puts_rows.append(curr_row)
                elif curr_option.contract.right == 'C':
                    calls_rows.append(curr_row)
            except:
                continue

        puts = pd.concat(puts_rows).reset_index(drop = True).sort_values(['Expiry','Strike'])
        calls = pd.concat(calls_rows).reset_index(drop = True).sort_values(['Expiry','Strike'])
        puts = time_remaining(puts, curr_sec_price)
        calls = time_remaining(calls, curr_sec_price)
        puts_list.append(puts)
        calls_list.append(calls)
        time.sleep(5)

    puts_df = pd.concat(puts_list,axis = 0).reset_index(drop = True)
    calls_df = pd.concat(calls_list,axis = 0).reset_index(drop = True)

    if save_csv:
        curr_time = dt.datetime.now().strftime('%Y%m%d-%H-%M')
        puts_df.to_csv('ib_puts' + curr_time + '.csv')
        calls_df.to_csv('ib_calls' + curr_time + '.csv')
        
    return puts_df, calls_df


def get_spreads(weekly_options, curr_spx, curr_vix, curr_skew, expiry_index = 0, rowshift = 1, commissions = 0.0266):

    expirations_dates = weekly_options.Expiry.drop_duplicates().tolist()
    
    weekly_options = weekly_options[weekly_options.Expiry == expirations_dates[expiry_index]]

    typ = weekly_options.reset_index().loc[0,'Type']
    
    if typ == 'P':
        weekly_options = weekly_options.sort_values('Strike', ascending = False).reset_index(drop = True)
    if typ == 'C':
        weekly_options = weekly_options.sort_values('Strike', ascending = True).reset_index(drop = True)
        
    time_remaining = weekly_options.reset_index(drop = True).loc[0,'time_remaining']

    shorts = weekly_options[['Bid', 'Strike', 'bidDelta', 'bidGamma', 'bidTheta', 'bidVega']]
    shorts.columns = ['Price','Strike','Delta','Gamma','Theta','Vega']
    longs = weekly_options[['Ask','Strike', 'askDelta', 'askGamma','askTheta', 'askVega']]
    longs.columns = ['Price','Strike','Delta','Gamma','Theta','Vega']

    spreads = shorts - longs.shift(-rowshift)
    spreads['Short_Strike'] = shorts.Strike
    spreads['Short_Delta'] = shorts.Delta
    spreads['Long_Strike'] = longs.Strike.shift(-rowshift)
    spreads = spreads[['Short_Strike', 'Long_Strike', 'Short_Delta', 'Price',
                       'Strike', 'Delta', 'Gamma', 'Theta', 'Vega']]

    spreads['Credit'] = spreads.Price - commissions

    if typ == 'P':
        spreads['MaxLoss'] = (-spreads.Strike + spreads.Credit)*100
        spreads['BreakEven'] = spreads['Short_Strike'] - spreads.Credit
    if typ == 'C':
        spreads['MaxLoss'] = (spreads.Strike + spreads.Credit)*100
        spreads['BreakEven'] = spreads['Short_Strike'] + spreads.Credit

    del spreads['Price'], spreads['Strike']

    spreads = spreads[spreads.Credit > 0].dropna().reset_index(drop = True)

    stepsize = 0.01
    period_iv = np.sqrt(time_remaining)*curr_vix/100
    
    skew_implied_2std = 0.027*(curr_skew - 100)/10
    skew_implied_3std = 0.006*(curr_skew - 100)/10

    period_iv_2std = (-2*period_iv)/norm.ppf(skew_implied_2std,0,1)
    period_iv_3std = (-3*period_iv)/norm.ppf(skew_implied_3std,0,1)

    spreads['EV'] = np.nan
    spreads['Win Prob'] = np.nan

    ev_lsts = []

    for idx, row in spreads.iterrows():

        if typ == 'P':
            ev_df = pd.DataFrame({'SPX': np.arange(row.Long_Strike, row.Short_Strike + stepsize, stepsize)})
            ev_df = ev_df[(ev_df['SPX'] < row.Short_Strike + stepsize)]
            ev_df['PnL'] = ev_df['SPX'] - row.Short_Strike + row.Credit
        if typ == 'C':
            ev_df = pd.DataFrame({'SPX': np.arange(row.Short_Strike, row.Long_Strike + stepsize, stepsize)})
            ev_df = ev_df[(ev_df['SPX'] < row.Long_Strike + stepsize)]
            ev_df['PnL'] = row.Short_Strike - ev_df['SPX'] + row.Credit

        ev_df['Prob'] = norm.cdf(ev_df['SPX']/curr_spx - 1,0, period_iv)
        ev_df[ev_df.SPX < curr_spx*(1 - period_iv)]['Prob'] = norm.cdf(ev_df[ev_df.SPX < curr_spx*(1 - period_iv)]['SPX']/curr_spx - 1,
                                                                       0, period_iv_2std)
        ev_df[ev_df.SPX < curr_spx*(1 - 2*period_iv)]['Prob'] = norm.cdf(ev_df[ev_df.SPX < curr_spx*(1 - 2*period_iv)]['SPX']/curr_spx - 1,
                                                                         0, period_iv_3std)
        
        ev_df[ev_df.SPX > curr_spx*(1 - period_iv)]['Prob'] = 1 - norm.cdf(ev_df[ev_df.SPX > curr_spx*(1 - period_iv)]['SPX']/curr_spx - 1,
                                                                       0, period_iv_2std)
        ev_df[ev_df.SPX > curr_spx*(1 - 2*period_iv)]['Prob'] = 1 - norm.cdf(ev_df[ev_df.SPX > curr_spx*(1 - 2*period_iv)]['SPX']/curr_spx - 1,
                                                                         0, period_iv_3std)
        
        lb = ev_df.loc[0,'Prob']
        ub = ev_df.loc[len(ev_df) - 1,'Prob']
        ev_df['Prob'] = ev_df['Prob'].diff()
        ev_df.loc[0,'Prob'] = lb
        ev_df.loc[len(ev_df) - 1, 'Prob'] = 1 - ub
        ev_df['EV'] = ev_df.Prob*ev_df.PnL

        ev_lsts.append(ev_df)
        total_ev = sum(ev_df['EV'])
        win_prob = sum(ev_df[ev_df['PnL'] >= 0]['Prob'])

        spreads.loc[idx, 'EV'] = total_ev
        spreads.loc[idx, 'Win Prob'] = win_prob
    
    spreads['Expiry'] = expirations_dates[expiry_index]
    spreads.Expiry = pd.to_datetime(spreads['Expiry'])
    return spreads, ev_lsts

In [ ]:
def weekly_atm_spreads(puts, curr_spx, curr_vix, curr_skew, expiry_index, 
                       maxshift = 6, commissions = 0.0266, max_contracts = 10):
    filtered_ps = []

    for rowshift in range(1,maxshift):

        put_spreads, evs = get_spreads(puts, curr_spx, curr_vix, curr_skew, expiry_index, rowshift, commissions)
        put_spreads = put_spreads[(put_spreads.Credit > 0.5) & 
                                  (abs(put_spreads.Short_Delta) < 0.6)].sort_values('EV', ascending = False)
        filtered_ps.append(put_spreads)

    filtered_ps = pd.concat(filtered_ps, axis = 0).reset_index(drop = True)

    atm_puts = puts[(puts.Expiry == filtered_ps.reset_index(drop = True).loc[0, 'Expiry']) &
                    (abs((abs(puts.modelDelta) - 0.5)) <= 0.05)][['Expiry','Ask','Bid',
                                                                  'Strike','Type',
                                                                  'askDelta','bidDelta',
                                                                  'modelDelta']].sort_values('modelDelta', 
                                                                                             ascending = False).reset_index(drop = True)
    atm_delta = atm_puts.loc[0, 'modelDelta']
    atm_credit = atm_puts.loc[0, 'Bid']

    weekly_ps = filtered_ps.copy().reset_index(drop = True)
    weekly_ps['ContractsForATMEquiv'] = round(pd.to_numeric(atm_delta/weekly_ps.Delta),0)

    conversion_columns = ['Delta','Gamma','Theta','Vega','Credit','MaxLoss','EV']

    for col in conversion_columns:
        weekly_ps[col] = weekly_ps[col]*weekly_ps.ContractsForATMEquiv

    weekly_ps = weekly_ps[(weekly_ps.Credit >= atm_credit + weekly_ps.ContractsForATMEquiv*commissions) &
                          (weekly_ps.ContractsForATMEquiv <= max_contracts)].reset_index(drop = True)
    return weekly_ps

def csp_hedges(calls, puts, curr_spx, curr_vix, curr_skew, expiry_index, 
               maxshift = 6, commissions = 0.0266):
    cs_hedge = []

    for rowshift in range(1,maxshift):

        call_spreads, evs = get_spreads(calls, curr_spx, curr_vix, curr_skew, expiry_index, rowshift, commissions)
        call_spreads = call_spreads[(call_spreads.Credit > 0.5)].sort_values('EV', ascending = False)
        cs_hedge.append(call_spreads)

    cs_hedge = pd.concat(cs_hedge, axis = 0).reset_index(drop = True)
    cs_hedge = cs_hedge[(cs_hedge.EV > 0)]

    expirations_dates = puts.Expiry.drop_duplicates().tolist()

    long_puts_hedge = puts[(puts.Expiry == expirations_dates[expiry_index])][['Ask', 'Bid', 'Expiry', 'Strike',
                                                                                    'modelDelta', 'modelGamma',
                                                                                    'modelTheta', 'modelVega', 'time_remaining']].reset_index(drop = True)
    long_puts_hedge['Mid'] = (long_puts_hedge['Ask'] + long_puts_hedge['Bid'])/2
    long_puts_hedge.columns = [x.replace('model','') for x in long_puts_hedge.columns.tolist()]

    time_remaining = long_puts_hedge.loc[0,'time_remaining']

    stepsize = 0.01
    period_iv = np.sqrt(time_remaining)*curr_vix/100

    skew_implied_2std = 0.027*(curr_skew - 100)/10
    skew_implied_3std = 0.006*(curr_skew - 100)/10

    period_iv_2std = (-2*period_iv)/norm.ppf(skew_implied_2std,0,1)
    period_iv_3std = (-3*period_iv)/norm.ppf(skew_implied_3std,0,1)

    hedges_lst = []

    for idx, row in cs_hedge.iterrows():

        curr_hedge = long_puts_hedge[long_puts_hedge.Mid < row.Credit][['Mid','Expiry', 'Strike',
                                                                        'Delta', 'Gamma',
                                                                        'Theta', 'Vega', 'time_remaining']]
        curr_hedge['Gamma'] = curr_hedge['Gamma'] - row.Gamma
        curr_hedge['Delta'] = curr_hedge['Delta'] - row.Delta
        curr_hedge['Theta'] = curr_hedge['Theta'] - row.Theta
        curr_hedge['Vega'] = curr_hedge['Vega'] - row.Vega
        curr_hedge = curr_hedge[(curr_hedge.Delta < 0) &
                                (curr_hedge.Gamma > 0) &
                                (curr_hedge.Theta > 0) &
                                (curr_hedge.Vega > 0)]
        curr_hedge['Credit'] = -curr_hedge.Mid + row.Credit - 0.0133
        curr_hedge['MaxLoss'] = (row.Short_Strike - row.Long_Strike + curr_hedge.Credit)*100
        curr_hedge['BreakEven'] = row.Short_Strike + curr_hedge['Credit']
        curr_hedge['Maturity'] = curr_hedge['Expiry']
        curr_hedge['Mid'] = row.Short_Strike
        curr_hedge['Expiry'] = row.Long_Strike
        curr_hedge.columns = ['Short_Call_Strike','Long_Call_Strike','Long_Put_Strike'] + curr_hedge.columns.tolist()[3:]

        curr_hedge['EV'] = np.nan
        curr_hedge['Win Prob'] = np.nan

        for idx_ch, row_ch in curr_hedge.iterrows():

            ev_df = pd.DataFrame({'SPX': np.arange(0, row_ch.Long_Call_Strike + stepsize, stepsize)})
            ev_df = ev_df[(ev_df['SPX'] < row_ch.Long_Call_Strike + stepsize)]
            ev_df['Prob'] = norm.cdf(ev_df['SPX']/curr_spx - 1,0, period_iv)

            ev_df[ev_df.SPX < curr_spx*(1 - period_iv)]['Prob'] = norm.cdf(ev_df[ev_df.SPX < curr_spx*(1 - period_iv)]['SPX']/curr_spx - 1,
                                                                           0, period_iv_2std)
            ev_df[ev_df.SPX < curr_spx*(1 - 2*period_iv)]['Prob'] = norm.cdf(ev_df[ev_df.SPX < curr_spx*(1 - 2*period_iv)]['SPX']/curr_spx - 1,
                                                                             0, period_iv_3std)

            ev_df[ev_df.SPX > curr_spx*(1 - period_iv)]['Prob'] = 1 - norm.cdf(ev_df[ev_df.SPX > curr_spx*(1 - period_iv)]['SPX']/curr_spx - 1,
                                                                               0, period_iv_2std)
            ev_df[ev_df.SPX > curr_spx*(1 - 2*period_iv)]['Prob'] = 1 - norm.cdf(ev_df[ev_df.SPX > curr_spx*(1 - 2*period_iv)]['SPX']/curr_spx - 1,
                                                                                 0, period_iv_3std)

            lb = ev_df.loc[0,'Prob']
            ub = ev_df.loc[len(ev_df) - 1,'Prob']
            ev_df['Prob'] = ev_df['Prob'].diff()
            ev_df.loc[0,'Prob'] = lb
            ev_df.loc[len(ev_df) - 1, 'Prob'] = 1 - ub
            ev_df['Long_Put_PnL'] = row_ch.Long_Put_Strike - ev_df.SPX
            ev_df.loc[ev_df['Long_Put_PnL'] < 0,'Long_Put_PnL'] = 0
            ev_df['Short_Call_PnL'] = ev_df.SPX - row_ch.Short_Call_Strike
            ev_df.loc[ev_df['Short_Call_PnL'] < 0,'Short_Call_PnL'] = 0
            ev_df['PnL'] = ev_df.Long_Put_PnL - ev_df.Short_Call_PnL + row_ch.Credit
            ev_df['EV'] = ev_df.Prob*ev_df.PnL

            total_ev = sum(ev_df['EV'])
            win_prob = sum(ev_df[ev_df['PnL'] >= 0]['Prob'])
            curr_hedge.loc[idx_ch, 'EV'] = total_ev
            curr_hedge.loc[idx_ch, 'Win Prob'] = win_prob

        hedges_lst.append(curr_hedge)

    hedges_df = pd.concat(hedges_lst, axis = 0)    
    return hedges_df


In [3]:
client_id = 13

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=client_id)

<IB connected to 127.0.0.1:7496 clientId=13>

In [4]:
spx = Index('SPX', 'CBOE')
vix = Index('VIX', 'CBOE')
skew = Index('SKEW', 'CBOE')
ib.qualifyContracts(skew)
ib.qualifyContracts(spx)

ib.reqHeadTimeStamp(spx, whatToShow='TRADES', useRTH=True)
ib.reqHeadTimeStamp(vix, whatToShow='TRADES', useRTH=True)

end_date = '' #'20100506 13:00:00 PST'
duration = '1 D'

spx_bars = ib.reqHistoricalData(
        spx,
        endDateTime=end_date,
        durationStr=duration,
        barSizeSetting='1 min',
        whatToShow='TRADES',
        useRTH=True,
        formatDate=1)

vix_bars = ib.reqHistoricalData(
        vix,
        endDateTime=end_date,
        durationStr=duration,
        barSizeSetting='1 min',
        whatToShow='TRADES',
        useRTH=True,
        formatDate=1)

spx_df = util.df(spx_bars).set_index('date')
spx_df.index = pd.to_datetime(spx_df.index)

vix_df = util.df(vix_bars).set_index('date')
vix_df.index = pd.to_datetime(vix_df.index)

curr_spx = spx_df.reset_index().loc[len(spx_df) - 1, 'close']

curr_vix = vix_df.reset_index().loc[len(vix_df) - 1, 'close']

curr_skew = ib.reqMktData(skew, '', False, False).marketPrice()

if math.isnan(curr_skew):
    skew = pd.read_csv('http://www.cboe.com/publish/scheduledtask/mktdata/datahouse/skewdailyprices.csv', 
                       skiprows = 1)[['Date','SKEW']].set_index('Date')
    curr_skew = skew.reset_index().loc[len(skew) - 1, 'SKEW']

chains = ib.reqSecDefOptParams(spx.symbol, '', spx.secType, spx.conId)

chains_df = util.df(chains)
chain = next(c for c in chains if c.tradingClass == 'SPXW' and c.exchange == 'SMART')

all_expirations = sorted(exp for exp in chain.expirations)

In [10]:
expirations

'20200331'

In [11]:
expirations = [all_expirations[-1]] #[all_expirations[1]] + 
expirations_dates = [dt.datetime.strptime(x, '%Y%m%d') for x in expirations]
#dte_expirations = list(filter(lambda x: x <= 10 and x > 0, [(expir - dt.datetime.today()).days for expir in expirations_dates]))
dte_expirations = [(expir - dt.datetime.today()).days for expir in expirations_dates]
#expirations = expirations[:len(dte_expirations)]

bounds_lst = [curr_spx*(5*curr_vix/(100*np.sqrt(365.25/dte))) for dte in dte_expirations]
bounds_lst = [curr_spx*(curr_vix/(100*np.sqrt(365.25/dte_expirations[0])))] + bounds_lst[1:]

In [12]:
contracts_list = []

for bounds, expiration in zip(bounds_lst, expirations):
    strikes = [strike for strike in chain.strikes
               if strike % 5 == 0
               and curr_spx - bounds < strike < curr_spx + bounds]
    
    rights = ['P', 'C']
    
    curr_contracts = [Option('SPX', expiration, strike, right, 'SMART')
                      for right in rights
                      for strike in strikes]
    contracts_list += [curr_contracts]
    
puts_list = []
calls_list = []

for contracts in contracts_list:
    ib.qualifyContracts(*contracts)
    spx_options = ib.reqTickers(*contracts)
    
    puts_rows = []
    calls_rows = []

    for curr_option in spx_options:
        try:
            curr_row = create_dfrow(curr_option)
            if curr_option.contract.right == 'P':
                puts_rows.append(curr_row)
            elif curr_option.contract.right == 'C':
                calls_rows.append(curr_row)
        except:
            continue

    puts = pd.concat(puts_rows).reset_index(drop = True).sort_values(['Expiry','Strike'])
    calls = pd.concat(calls_rows).reset_index(drop = True).sort_values(['Expiry','Strike'])
    puts = time_remaining(puts, curr_spx)
    calls = time_remaining(calls, curr_spx)
    puts_list.append(puts)
    calls_list.append(calls)
    time.sleep(5)

puts_df = pd.concat(puts_list,axis = 0).reset_index(drop = True)
calls_df = pd.concat(calls_list,axis = 0).reset_index(drop = True)

curr_time = dt.datetime.now().strftime('%Y%m%d-%H-%M')
puts_df.to_csv('ib_puts' + curr_time + '.csv')
calls_df.to_csv('ib_calls' + curr_time + '.csv')


Started to throttle requests
Error 200, reqId 13: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2560.0, right='P', exchange='SMART')
Error 200, reqId 15: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2565.0, right='P', exchange='SMART')
Error 200, reqId 16: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=3080.0, right='P', exchange='SMART')
Error 200, reqId 17: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2570.0, right='P', exchange='SMART')
Error 200, reqId 20: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=3090.0, right='P', exchange='SMART')
Error 200, 

Error 200, reqId 70: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2745.0, right='P', exchange='SMART')
Error 200, reqId 72: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2755.0, right='P', exchange='SMART')
Error 200, reqId 73: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2760.0, right='P', exchange='SMART')
Error 200, reqId 74: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2765.0, right='P', exchange='SMART')
Error 200, reqId 75: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2770.0, right='P', exchange='SMART')
Error 200, reqId 78: No security definit

Error 200, reqId 124: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=3015.0, right='P', exchange='SMART')
Error 200, reqId 125: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=3020.0, right='P', exchange='SMART')
Error 200, reqId 127: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=3030.0, right='P', exchange='SMART')
Error 200, reqId 129: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=3035.0, right='P', exchange='SMART')
Error 200, reqId 128: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2520.0, right='P', exchange='SMART')
Error 200, reqId 131: No security d

Error 200, reqId 179: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2660.0, right='C', exchange='SMART')
Error 200, reqId 181: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2665.0, right='C', exchange='SMART')
Error 200, reqId 182: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=3180.0, right='C', exchange='SMART')
Error 200, reqId 185: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2680.0, right='C', exchange='SMART')
Error 200, reqId 183: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2670.0, right='C', exchange='SMART')
Error 200, reqId 186: No security d

Error 200, reqId 234: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2910.0, right='C', exchange='SMART')
Error 200, reqId 235: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2915.0, right='C', exchange='SMART')
Error 200, reqId 236: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2920.0, right='C', exchange='SMART')
Error 200, reqId 238: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2930.0, right='C', exchange='SMART')
Error 200, reqId 239: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2935.0, right='C', exchange='SMART')
Error 200, reqId 240: No security d

...

Error 200, reqId 284: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=3095.0, right='P', exchange='SMART')
Error 200, reqId 285: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2585.0, right='P', exchange='SMART')
Error 200, reqId 287: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2590.0, right='P', exchange='SMART')
Error 200, reqId 288: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=3105.0, right='P', exchange='SMART')
Error 200, reqId 289: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2595.0, right='P', exchange='SMART')
Error 200, reqId 290: No security d

Error 200, reqId 341: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2790.0, right='P', exchange='SMART')
Error 200, reqId 342: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2795.0, right='P', exchange='SMART')
Error 200, reqId 344: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2805.0, right='P', exchange='SMART')
Error 200, reqId 345: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2810.0, right='P', exchange='SMART')
Error 200, reqId 346: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2815.0, right='P', exchange='SMART')
Error 200, reqId 347: No security d

Error 200, reqId 395: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=3045.0, right='P', exchange='SMART')
Error 200, reqId 396: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2535.0, right='P', exchange='SMART')
Error 200, reqId 398: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2540.0, right='P', exchange='SMART')
Error 200, reqId 399: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=3055.0, right='P', exchange='SMART')
Error 200, reqId 400: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2545.0, right='P', exchange='SMART')
Error 200, reqId 401: No security d

Error 200, reqId 451: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2695.0, right='C', exchange='SMART')
Error 200, reqId 453: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2705.0, right='C', exchange='SMART')
Error 200, reqId 454: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2710.0, right='C', exchange='SMART')
Error 200, reqId 455: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=3225.0, right='C', exchange='SMART')
Error 200, reqId 456: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2715.0, right='C', exchange='SMART')
Error 200, reqId 457: No security d

Error 200, reqId 506: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2960.0, right='C', exchange='SMART')
Error 200, reqId 507: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2965.0, right='C', exchange='SMART')
Error 200, reqId 508: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2970.0, right='C', exchange='SMART')
Error 200, reqId 510: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2980.0, right='C', exchange='SMART')
Error 200, reqId 511: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200331', strike=2985.0, right='C', exchange='SMART')
Error 200, reqId 512: No security d

In [ ]:
expiry_index = 0

weekly_ps = weekly_atm_spreads(puts_df, curr_spx, curr_vix, curr_skew, expiry_index, 
                               maxshift = 6, commissions = 0.0266, max_contracts = 10)

weekly_ps['VegaToTheta'] = abs(weekly_ps.Vega/weekly_ps.Theta)

In [ ]:
weekly_ps.sort_values('VegaToTheta')

In [19]:
ib.disconnect()

In [ ]:
calls_df

In [ ]:
expiry_index = 0
maxshift = 6

csp_sept = csp_hedges(calls_df, puts_df, curr_spx, curr_vix, curr_skew, expiry_index, 
                      maxshift = 6, commissions = 0.0266)

In [ ]:
csp_sept[(csp_sept['EV'] > 0) & 
         (csp_sept['Win Prob'] > 0.5) &
         (csp_sept['MaxLoss'] > -5000) &
         (csp_sept['Short_Call_Strike'] > 2850)].sort_values('Vega', ascending = False)
